<a href="https://colab.research.google.com/github/protocol-streams/querent-experimental/blob/main/BERT_Relationship_Extraction_with_Zeroshot_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://chat.openai.com/share/a0b59daa-d1fa-4e68-bb67-0c252a71abd3

In [1]:
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.7 MB/s eta 0:00:00


In [3]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load pre-trained BERT model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

def extract_relationship(document, entity1, entity2, max_length=512):
    """
    Extracts the relationship between two entities in a given document.

    Parameters:
    - document (str): The text/document in which to find the relationship.
    - entity1 (str): The first entity.
    - entity2 (str): The second entity.
    - max_length (int): The maximum token length for BERT input. Default is 512.

    Returns:
    - str: The extracted relationship or 'Unknown' if not found.
    """

    # Frame the relationship extraction as a question
    question = f"What is the relationship between {entity1} and {entity2}?"

    # Tokenize input
    inputs = tokenizer.encode_plus(question, document, add_special_tokens=True, max_length=max_length, truncation=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    # Get answer from BERT
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Convert tokens to string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    return answer if answer else "Unknown"

# Example usage:
document = "Apple Inc. is an American multinational technology company that designs, manufactures, and markets smartphones, including the iPhone. Steve Jobs was one of the co-founders of Apple."
entity_pairs = [("Apple Inc.", "iPhone"), ("Steve Jobs", "Apple Inc.")]

for entity1, entity2 in entity_pairs:
    relationship = extract_relationship(document, entity1, entity2, max_length=256)  # Here, we set max_length to 256 as an example.
    print(f"Relationship between {entity1} and {entity2}: {relationship}")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Relationship between Apple Inc. and iPhone: apple inc . is an american multinational technology company that designs , manufactures , and markets smartphones , including the iphone
Relationship between Steve Jobs and Apple Inc.: steve jobs was one of the co - founders of apple
